In [1]:
import logging
import pickle
import time
from statistics import mean, median

import en_core_web_sm
import gensim.utils
import gensim.models.keyedvectors
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
import seaborn as sns
import spacy
import string

from glove import Glove, Corpus
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from imblearn.under_sampling import RandomUnderSampler
from keras_preprocessing.sequence import pad_sequences

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
nltk.download('punkt')

logging.basicConfig(format="%(message)s")
logger = logging.getLogger()

ImportError: cannot import name 'Corpus' from 'glove' (C:\Users\Kika\anaconda3\lib\site-packages\glove\__init__.py)

# 5 - Model Word2Vec

### 5.1 - Načítanie predspracovaných viet

In [2]:
#Vytvorenie vektorov pre NEUPRAVENÉ VETY = 0
#Vytvorenie vektorov pre UPRAVENÉ VETY (o synonymá) = 1

file = 0
if file == 0:
    file = "sentences"
else:
    file = "sentences_new"

In [3]:
#Načítanie viet
with open("files/"+file, "rb") as fp:   # Unpickling
    sentences = pickle.load(fp)

### 5.2 - Vytvorenie slovných vektorov

In [4]:
dimension_size = 200
window_size = 7


embeddings_index = dict()

start = time.time()
#Tréning slovných vektorov

corpus = Corpus()
corpus.fit(sentences, window=window_size)
glove = Glove(
            no_components=dimension_size,      # Veľkosť dimenzie výsledných slovných vektorov
            learning_rate=0.05)                # Rýchlosť učenia
glove.fit(corpus.matrix, epochs=30, no_threads=4)
glove.add_dictionary(corpus.dictionary)

words = set(list(glove.dictionary.keys()))
for word in words:
    embeddings_index[word] = glove.word_vectors[glove.dictionary[word]]
    
end = time.time()

In [5]:
print("Count of words:",len(words),", training time:",(end-start))

Count of words: 25735 , training time: 33.24569869041443


### 5.3 - Uloženie vektorov do BIN súboru

In [6]:
# konvertovanie slovníka do formátu KeyedVectors
kv = KeyedVectors(vector_size=dimension_size)
kv.add_vectors(list(embeddings_index.keys()), list(embeddings_index.values()))

# uloženie slovníka do binárneho súboru
kv.save_word2vec_format("files/model_GloVe_"+"dim_"+str(dimension_size)+"_"+time.strftime("%Y%m%d%H%M")+"_"+file+".bin", binary=True)

attribute count not present in KeyedVectors<vector_size=200, 25735 keys>; will store in internal index_to_key order


In [8]:
#Príklad pre podobnosť
word2vec.wv.most_similar("plane")

[('flight', 0.9181925058364868),
 ('jet', 0.9180074334144592),
 ('passenger', 0.91603022813797),
 ('bomb', 0.9080346822738647),
 ('crashed', 0.9069170951843262),
 ('airport', 0.9053643345832825),
 ('pilot', 0.9050325155258179),
 ('crash', 0.9040655493736267),
 ('train', 0.9030295014381409),
 ('burned', 0.9005767703056335)]